#MNIST Fashion




La clasificación es ampliamente usada en inteligencia artificial, en varios campos. Existen varias maneras de construir clasificadores usando técnicas de machine learning y deep learning.

Esta guia usa [tf.keras](https://www.tensorflow.org/guide/keras), una API de alto nivel para construir y entrenar modelos

## Instalar e importar dependencias

Usaremos un conjunto de datos de la página [TensorFlow Datasets](https://www.tensorflow.org/datasets/). Esta API simplifica el descargar y acceder los datos. También provee muchos conjuntos de datos. También usaremos algunas librerías auxiliares.

In [ ]:
!pip install -U tensorflow_datasets

Importamos las dependencias:

In [ ]:
# Import Tensorflow
import tensorflow as tf
# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

Ademas, configuramos la salida de tensorflow para que solo nos muestre los errores

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

## Importar el Fashion MNIST dataset

Esta guia usa el set de datos [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), el cual contiene 70.000 imagenes en blanco y negro en 10 categorías. Cada imagen es un articulo de ropa de tamaño (28 $\times$ 28 pixeles), como se muestra a continuación:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Imágenes del Fashion MNIST" width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figura 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Usaremos 60.000 imagenes para entrenar la red neuronal y 10.000 imagenes para evaluar el desempeño de la red. Puedes acceder al set de datos directamente, usando la [API](https://www.tensorflow.org/datasets).

La siguiente linea importa y desempaqueta el conjunto de datos. En este caso, la función recibe los siguientes parámetros:

*   `name="fashion_mnist"`: El nombre del conjunto de datos que queremos descargar. Este nombre tiene que ser exacto para que se pueda encontrar el conjunto de datos. Puedes ver los conjuntos de datos y sus nombres [aquí](https://www.tensorflow.org/datasets).
*   `as_supervised=True`: El conjunto de datos obtenido tendrá una estructura de tupla (input, label). Si `as_supervised` fuera `False`, el conjunto de datos obtenido tendría un diccionario con todas las características.

*   `with_info=True`: Si es `True`, la función retornará una tupla donde el primer elemento sería la información del conjunto de datos y el segundo la metadata relevante al set de datos.

Mas argumentos pueden ser encontrados en la siguiente [documentación](https://www.tensorflow.org/datasets/api_docs/python/tfds/load)



In [ ]:
dataset, metadata = tfds.load(name='fashion_mnist', as_supervised=True, with_info=True)

Dividimos el conjunto de datos en entrenamiento (training set) y prueba (test set). Dijimos que dividiríamos el conjunto de datos en 60.000 imágenes de entrenamiento y 10.000 de prueba. No necesitaremos hacer esto último debido a que el conjunto de datos ya se encuentra dividido.


In [ ]:
train_dataset, test_dataset = dataset['train'], dataset['test']

La línea de código anterior carga el conjunto de datos y retorna el metadata junto con los conjuntos de datos de entrenamiento y pruebas.

* El modelo es entrenado utilizando el `train_dataset`.
* El modelo es probado con el `test_dataset`.

Las imagenes son arreglos de 28 $\times$ 28 pixeles. Cada pixel está en el rango `[0, 255]` en tonalidad que va del blanco al negro.

Los `labels` o etiquetas de las imágenes son arreglos de enteros, en el rango `[0, 9]`. Estos corresponden a la clase de prenda que la imagen representa:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>Polera/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Pantalón</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Chaleco</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Vestido</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Abrigo</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandalia</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Camisa</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Zapatilla</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bolso</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Bota</td>
  </tr>
</table>

Cada imagen corresponde a una clase en específico. Como los nombres de las clases no están incluidos en el set de datos, los guardamos para mostrarlos después.

In [ ]:
class_names = ['Polera/top', 'Pantalon', 'Chaleco', 'Vestido', 'Abrigo',
               'Sandalia', 'Camisa',   'Zapatilla',  'Bolso',   'Bota']

### Explorar los datos

Exploremos el formato de los datos antes de entrenar el modelo. El bloque de codigo a continuación nos muestra que hay 60.000 imágenes en el conjunto de entrenamiento y 10.000 imágenes en el conjunto de pruebas.

In [ ]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

## Preprocesar los datos

El valor de cada pixel en la imágen es un entero en el rango `[0, 255]`. Para que el modelo funcione como corresponde, normalizaremos estos datos al rango `[0,1]`. Creamos una función de normalización y la aplicamos a cada imagen de cada conjunto de datos.

In [ ]:
#NORMALIZADO
# Definir la función de normalización
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Aplicar la normalización usando map y luego configurar el batching y el caching
train_dataset_norm = train_dataset.map(normalize)
test_dataset_norm = test_dataset.map(normalize)


Mostramos las primeras 25 imágenes del conjunto de pruebas y mostramos el nombre de la clase abajo de cada imagen. También verificamos que los datos estén correctos y estaremos listos para armar y entrenar la red.

In [ ]:
# Tomamos una imagen y le quitamos la dimensíon del color haciendo un reshape
image, label = tf.data.experimental.get_single_element(test_dataset_norm.take(1))

image = image.numpy().reshape((28,28))
# Mostramos la imagen
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()


## Construir el modelo

Construir el modelo requiere configurar las capas del modelo y luego compilarlo.

### Configurar las capas

El bloque más básico de construcción de un modelo es una capa. Esta extrae la representación de la data que se le introdujo. Una serie de capas conectadas creará una representación significativa para el problema.

Mucho del deep learning consiste en juntar varias capas simples. Varias capas como la `tf.keras.layers.Dense`, tienen parámetros internos que son ajustados mientras se ejecuta el entrenamiento

Creamos dos modelos para diferencia entre datos normalizados y sin normalizar

In [ ]:
model_no_norm = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
model_norm = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

Esta red tiene 3 capas:

* **Capa de Entrada:** `tf.keras.layers.Flatten` — Esta capa transforma las imágenes de un arreglo bidimensional de 28 $\times$ 28 pixels a un arreglo unidemensional de 784 pixeles (28\*28). Esta capa no tiene parámetros para ajustar ya que, solo reforma los datos.


* **Capa "Oculta":** `tf.keras.layers.Dense`— Una capa densa conectada completamente de 128 neuronas. Cada neurona (o nodo) toma la entrada de los 784 nodos de la capa anterior y ajusta los pesos de acuerdo a los parámetros internos de cada neurona para luego pasar el resultado a la siguiente capa. Aquí la función de activación es una Relu.

* **Capa de Salida:**  `tf.keras.layers.Dense` — Una capa compuesta de 10 neuronas, las cuales representan cada clase. Cada neurona toma la salida de todos los nodos anteriores para representar una probabilidad, entre 0 y 1, para cada clase, a través del Softmax. La suma de la salida de todas las neuronas es 1.



### Compilar el modelo
Antes de que el modelo esté listo para el entrenamiento, necesita unas configuraciones adicionales.

* **Optimizador:** Un algoritmo para ajustar los parametros internos durante la ejecución del entrenamiento. Para más información revisa [Optimizers documentation](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam). En este caso se utilizam "Adam".
* **Función de pérdida:** Un algoritmo para cuantificar que tan bien está funcionando la red durante el entrenamiento, es decir, cuanto difieren los resultados actuales de los esperados (entre menos mejor). Para más información revisa [Losses documentation](https://www.tensorflow.org/api_docs/python/tf/keras/losses). En este caso se utiliza "Cross Entropy".
* **Métricas:** Usadas para monitorear el entrenamiento. Por ejemplo aquí se utilizará el "Accuracy". Para más información revisa [Metrics documentation](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [ ]:
model_no_norm.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model_norm.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## Entrenar el modelo

Primero, definimos el comportamiento de las iteraciones para el conjunto de entrenamiento:
1. Repetir para siempre especificando `dataset.repeat()`. El parámetro `epochs` define cuantas iteraciones del entrenameinto queremos hacer.
2. `dataset.shuffle(60000)` aleatoriza el orden en que son entregados los valores en cada iteración. De esta manera, el modelo no puede aprender nada del orden de los datos.
3. `dataset.batch(32)` le dice a `model.fit` que use conjuntos de 32 imágenes (batch) y etiquetas cuando se actualizen las variables del modelo. Este número puede ser cambiado dependiendo de las capacidades de nuestro hardware.

El entrenamiento es realizado llamando `model.fit`:
1. Dar el los datos de entrenamiento usando el conjunto de entrenamiento.
2. El modelo aprende a asociar imágenes con etiquetas ajustando los parámetros internos de cada una de las neuronas y sus sesgos (*biases*).
3. `epoch=5` limita el entrenamiento a 5 iteraciones completas del conjunto de entrenamiento. Entonces, pasarán un total de 5 * 60000 = 300000 imágenes.



A medida que el modelo se entrena, la pérdida y el accuracy son mostrados. Este modelo alcanza un accuracy de 0.89 aproximadamente, usando el conjunto de entrenamiento.

## Evaluar el accuracy

A continuación, comparamos como el modelo se comporta con el conjunto de pruebas (test set). Usaremos todos los ejemplos para obtener la métrica.

SIN NORMALIZAR

In [ ]:
BATCH_SIZE = 32
train_dataset_no_norm = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset_no_norm = test_dataset.cache().batch(BATCH_SIZE)


In [ ]:
model_no_norm.fit(train_dataset_no_norm, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

In [ ]:
test_loss, test_accuracy = model_no_norm.evaluate(test_dataset_no_norm, steps=math.ceil(num_test_examples/32))
print('Accuracy en el set de pruebas:', test_accuracy)

NORMALIZADO

In [ ]:
BATCH_SIZE = 32
train_dataset_norm = train_dataset_norm.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset_norm = test_dataset_norm.cache().batch(BATCH_SIZE)

In [ ]:
model_norm.fit(train_dataset_norm, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

In [ ]:
test_loss, test_accuracy = model_norm.evaluate(test_dataset_norm, steps=math.ceil(num_test_examples/32))
print('Accuracy en el set de pruebas:', test_accuracy)

Como podemos darnos cuenta, el accuracy en el conjunto de pruebas es menor que el del conjunto de entrenamiento. Esto es completamente normal, debido a que el modelo fue entrenado con el conjunto de entrenamiento. Cuando el modelo ve imágenes que nunca ha visto, podemos esperar que el rendimiento de la red baje.

## Hacer predicciones y explorar

Con el modelo entrenado, podemos utilizarlo para hacer predicciones sobre algunas imágenes.

In [ ]:
for test_images, test_labels in test_dataset_norm.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model_norm.predict(test_images)

In [ ]:
predictions.shape


Aqui, el modelo ha predicho una etiqueta para cada imagen en el conjunto de pruebas. Veamos la primera predicción.

In [ ]:
predictions[0]


Una predicción es un arreglo de 10 números. Estos describen la "confianza" del modelo de que esa imagen corresponde a cada una de las clases. Podemos ver cual es la clase con la mayor confianza:

In [ ]:
np.argmax(predictions[0])

El modelo tiene la mayor confianza en que esta es un abrigo, o `class_names[4]`. Y podemos ver la etiqueta que corresponde, a ver si es correcta.

In [ ]:
test_labels[0]

Podemos graficar esto para ver la ver el conjunto completo de las 10 clases.

In [ ]:
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Veamos la primera imagen, sus predicciones y el arreglo de predicciones.

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

Grafiquemos varias imágenes con sus predicciones. Las predicciones correctas son azules y las incorrectas: rojas. El número muestra la confianza, en porcentaje, de la etiqueta predicha. Es importante notar que puede estar equivocada aún con mucha confianza.

In [ ]:
# Imprime las primeras X imagenes de prueba, su etiqueta predicha y la etiqueta correcta
# Predicciones correcta en azul y las incorrectas en rojo
num_rows = 6
num_cols = 4
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
